In [1]:
import cv2
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


directory = 'Otherdata/' 

if not os.path.exists(directory): 
    os.mkdir(directory)  


for char in ['blank'] + [chr(i) for i in range(65, 91)]: 
    char_dir = os.path.join(directory, char)  
    if not os.path.exists(char_dir): 
        os.mkdir(char_dir)  


cap = cv2.VideoCapture(0)  

print("Press the corresponding key ('A'-'Z', '.') for the sign or 'q' to quit.") 
5
while True:
   
    _, frame = cap.read()  
    if not _:  
        print("Failed to capture frame. Exiting...") 
        break 

    
    cv2.rectangle(frame, (0, 40), (300, 300), (255, 255, 255), 2) 
    cv2.imshow("Data Collection", frame) 

    
    roi = frame[40:300, 0:300]  # Croping the frame (height: 40-300, width: 0-300)
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY) 
    roi_resized = cv2.resize(roi_gray, (128, 128))  

    
    key = cv2.waitKey(1) & 0xFF  # Wait for a key press.

   
    if key == ord('q'): 
        print("Exiting...") 
        break  

    # Capture and save the image if a valid key is pressed
    char = chr(key).upper() if key != ord('.') else 'blank' 
   
    if char in [chr(i) for i in range(65, 91)] + ['blank'] + ['Yoo']:
        char_dir = os.path.join(directory, char) 
        image_count = len(os.listdir(char_dir))  
        image_path = os.path.join(char_dir, f"{image_count}.jpg") 
        cv2.imwrite(image_path, roi_resized) 
        print(f"Image saved: {image_path}")  


cap.release()  
cv2.destroyAllWindows() 


Press the corresponding key ('A'-'Z', '.') for the sign or 'q' to quit.
Exiting...


In [12]:
#Training Part

directory = 'Otherdata/'


labels = ['blank'] + [chr(i) for i in range(65, 91)]

# Initialize lists to hold image data and corresponding labels
data = []
labels_data = []

# Load images from the directories and preprocess
for label in labels:
    label_dir = os.path.join(directory, label)
    
    if os.path.exists(label_dir):
        for img_name in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_name)
            
            # Load the image, convert to grayscale, and resize it
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (128, 128))  # Resize to match model input
            img = img.astype('float32') / 255.0  # Normalize the pixel values
            
            # Append image data and its label
            data.append(img)
            labels_data.append(labels.index(label))  # Numeric label for classification

# Convert the data and labels to numpy arrays
data = np.array(data)
labels_data = np.array(labels_data)

# Reshape data to match model input: (samples, height, width, channels)
data = data.reshape((data.shape[0], 128, 128, 1))


X_train, X_temp, y_train, y_temp = train_test_split(data, labels_data, test_size=0.7, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=2)

# Build the model
model = Sequential()

# Convolutional Layers with Max Pooling and Dropout for regularization
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels), activation='softmax'))  # Output layer with softmax activation

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, 
          epochs=150, 
          batch_size=32, 
          validation_data=(X_val, y_val), 
          callbacks=[early_stopping])

# Save the model architecture to a JSON file
model_json = model.to_json()
with open("sign_language_modelnew.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights to an H5 file
model.save_weights("sign_language_modelnew.h5")

print("Model training completed and saved.")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 1/150
132/132 [==============================] - 10s 65ms/step - loss: 3.2930 - accuracy: 0.0479 - val_loss: 3.2920 - val_accuracy: 0.0601
Epoch 2/150
132/132 [==============================] - 8s 58ms/step - loss: 3.0233 - accuracy: 0.1357 - val_loss: 2.6326 - val_accuracy: 0.2836
Epoch 3/150
132/132 [==============================] - 8s 58ms/step - loss: 2.1396 - accuracy: 0.3630 - val_loss: 1.4805 - val_accuracy: 0.5640
Epoch 4/150
132/132 [==============================] - 8s 58ms/step - loss: 1.5200 - accuracy: 0.5204 - val_loss: 1.1446 - val_accuracy: 0.6563
Epoch 5/150
132/132 [==============================] - 8s 58ms/step - loss: 1.1842 - accuracy: 0.6165 - val_loss: 0.9290 - val_accuracy: 0.7086
Epoch 6/150
132/132 [==============================] - 8s 58ms/step - loss: 0.9848 - accuracy: 0.6718 - val_loss: 0.7128 - val_accuracy: 0.7998
Epoch 7/150
132/132 [==============================] - 8s 58ms/step - loss: 0.8540 - accuracy: 0.7129 - val_loss: 0.6869 - val_accuracy

In [1]:
from keras.models import model_from_json
import cv2
import numpy as np

# Load model architecture
json_file_path = "sign_language_modelnew.json"  # Path to the .json file
weights_file_path = "sign_language_modelnew.h5"  # Path to the .h5 weights file

# Load JSON and create model
with open(json_file_path, "r") as json_file:
    model_json = json_file.read()
model = model_from_json(model_json)

# Load weights into the model
model.load_weights(weights_file_path)
print("Model loaded successfully.")

# Function to preprocess the image
def preprocess_image(image):
    feature = np.array(image, dtype="float32")
    feature = feature.reshape(1, 128, 128, 1)  # Ensure the input shape matches the model
    return feature / 255.0  # Normalize pixel values

# Labels for classification
label = ['blank'] + [chr(i) for i in range(65, 91)]  # 'blank' + 'A'-'Z'

# Initialize webcam
cap = cv2.VideoCapture(0)
print("Press 'q' to exit.")

while True:
    # Read frame from webcam
    _, frame = cap.read()
    if not _:
        print("Failed to capture frame.")
        break

    # Draw rectangle for ROI
    cv2.rectangle(frame, (0, 40), (300, 300), (0, 165, 255), 2)

    # Crop and preprocess the ROI
    roi = frame[40:300, 0:300]
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi_resized = cv2.resize(roi_gray, (128, 128))
    processed_roi = preprocess_image(roi_resized)

    # Make prediction
    pred = model.predict(processed_roi, verbose=0)
    prediction_label = label[pred.argmax()]
    accuracy = np.max(pred) * 100

    # Display prediction
    display_text = f"{prediction_label} ({accuracy:.2f}%)" if prediction_label != 'blank' else "No Sign"
    cv2.rectangle(frame, (0, 0), (300, 40), (0, 165, 255), -1)
    cv2.putText(frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Show the output
    cv2.imshow("Sign Language Recognition", frame)

    # Exit on 'q' key
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Model loaded successfully.
Press 'q' to exit.
